In [1]:
### GEOCODE DATAFRAME WITH GEOPY
# cf : https://geopy.readthedocs.io/en/stable/#installation
# cf : https://wiki.openstreetmap.org/wiki/Nominatim
# cf : http://blog.adrienvh.fr/2015/01/18/geocoder-en-masse-plusieurs-milliers-dadresses-avec-python-et-nominatim/

from time import sleep

import pprint
pp = pprint.PrettyPrinter(indent=2)

import pandas as pd

# cf : https://github.com/jmcarpenter2/swifter
# cf : https://medium.com/@jmcarpenter2/swiftapply-automatically-efficient-pandas-apply-operations-50e1058909f9
import swifter

from geopy.geocoders import Nominatim, BANFrance
from geopy.extra.rate_limiter import RateLimiter

from functools import partial
from tqdm import tqdm

In [2]:
tqdm.pandas()

**GENERIC VARIABLES**

In [3]:
dft_delay        = 1.5
dft_timeout      = 20
full_address_col = "_full_address_"
location_col     = "_location_"

**GENERIC FUNCTIONS**

In [4]:
geocoder_nom = Nominatim(user_agent="_solidata_app_")
geocoder_ban = BANFrance(user_agent="_solidata_app_")

In [5]:
### rate limiter
geocode_nom = RateLimiter(geocoder_nom.geocode, min_delay_seconds=dft_delay)
geocode_ban = RateLimiter(geocoder_ban.geocode, min_delay_seconds=dft_delay)

In [6]:
def LocToDict(location) : 
    if location != None : 
        return {
            "raw"       : location.raw,
            "address"   : location.address,
            "point"     : location.point,
            "latitude"  : location.latitude,
            "longitude" : location.longitude,
        }
    else : 
        return {
            "raw"        : None,
            "address"    : None,
            "point"      : None,
            "latitude"   : None,
            "longitude"  : None,
        }

In [7]:
### concat function
def concat_cols(row, columns_to_concat):
    if len(columns_to_concat) > 1 :
        return ", ".join( row[col] for col in columns_to_concat )
    else : 
        return row[columns_to_concat[0]]

In [34]:
### main geolocalizing function for dataframe
def geoloc_df( row, col_name, complement, time_out, delay ) : 

    print ("\n- row[col_name] : ", row[col_name])

    adress = ", ".join( [row[col_name], complement ] )
    print ("- adress : ", adress)
    
    try :
        location = geocoder_nom.geocode( query=adress, timeout=time_out, extratags=True)
    except : 
        location = geocoder_ban.geocode( query=adress, timeout=time_out)
        
    print ("- location : ", location)
    
    sleep(delay)
    
    if location : 
        return LocToDict(location)
    else : 
        return None
    

In [39]:
### main geolocalizing function for dataframe
def geoloc_df_col( 
    row_val, 
    complement="", 
    time_out=dft_timeout, 
    delay=dft_delay 
    ) : 

    print ("\n- row_val : ", row_val)
    
    if pd.notnull(row_val) : 
        
        adress = ", ".join( [ row_val, complement ] )
        print ("- adress : ", adress)

        try :
            location = geocoder_nom.geocode( query=adress, timeout=time_out, extratags=True)
        except : 
            location = geocoder_ban.geocode( query=adress, timeout=time_out)

        print ("- location : ", location)

        sleep(delay)

        if location : 
            return LocToDict(location)
        
        else : 
            return None
    
    else : 
        return None

**test with simple address**

In [9]:
### variables to geocode

adresse_01 = "191 rue Saint-Jacques, Paris, France"
adresse_02 = "ABERGEMENT CLEMENCIAT, 01001"

adress_complement = "France"

In [10]:
location_01 = geocoder_nom.geocode(query=adresse_01, timeout=dft_timeout, extratags=True)
print(location_01)

Institut de Géographie, 191, Rue Saint-Jacques, Val-de-Grâce, 5e, Paris, Île-de-France, France métropolitaine, 75005, France


In [11]:
location_01_dict = LocToDict(location_01)
pp.pprint(location_01_dict)

{ 'address': 'Institut de Géographie, 191, Rue Saint-Jacques, Val-de-Grâce, '
             '5e, Paris, Île-de-France, France métropolitaine, 75005, France',
  'latitude': 48.84468875,
  'longitude': 2.34258870005006,
  'point': Point(48.84468875, 2.34258870005006, 0.0),
  'raw': { 'boundingbox': [ '48.8445388',
                            '48.8448244',
                            '2.3423236',
                            '2.3428539'],
           'class': 'amenity',
           'display_name': 'Institut de Géographie, 191, Rue Saint-Jacques, '
                           'Val-de-Grâce, 5e, Paris, Île-de-France, France '
                           'métropolitaine, 75005, France',
           'extratags': { 'internet_access': 'no',
                          'wikidata': 'Q3152006',
                          'wikipedia': 'fr:Institut de géographie'},
           'icon': 'https://nominatim.openstreetmap.org/images/mapicons/education_university.p.20.png',
           'importance': 0.827646119222430

In [12]:
location_02a = geocoder_nom.geocode(adresse_02, True, dft_timeout)
print(location_02a)

None


In [13]:
location_02b = geocoder_ban.geocode(adresse_02, timeout=dft_timeout)
print((location_02b.latitude, location_02b.longitude))

(46.150598, 4.920347)


In [14]:
location_02_dict = LocToDict(location_02b)
pp.pprint(location_02_dict)

{ 'address': "L'Abergement-Clémenciat",
  'latitude': 46.150598,
  'longitude': 4.920347,
  'point': Point(46.150598, 4.920347, 0.0),
  'raw': { 'geometry': {'coordinates': [4.920347, 46.150598], 'type': 'Point'},
           'properties': { 'adm_weight': 1,
                           'city': "L'Abergement-Clémenciat",
                           'citycode': '01001',
                           'context': '01, Ain, Auvergne-Rhône-Alpes '
                                      '(Rhône-Alpes)',
                           'id': '01001',
                           'importance': 0.075,
                           'label': "L'Abergement-Clémenciat",
                           'name': "L'Abergement-Clémenciat",
                           'population': 0.8,
                           'postcode': '01400',
                           'score': 0.6613636363636364,
                           'type': 'municipality',
                           'x': 848200,
                           'y': 6563000},
        

**test with pandas**

In [15]:
csv_test_01 = "tests_jupyter/2016_codes_postaux_prep.csv"
csv_test_02 = "tests_jupyter/export_openscraper_5aa3de630a828651b2ebfe3e.csv"

In [16]:
df_01 = pd.read_csv(csv_test_01, encoding="utf8")
df_01.head(3)

,Code_commune_INSEE,Nom_commune,Code_postal,Libelle_acheminement,Ligne_5,latitude,longitude,Population,Surface,geopoint_2d
0,01001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.156818,4.924707,767.0,1567.0,POINT(4.924707496412453 46.15681799286022)
1,01002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.010085,5.428759,236.0,909.0,POINT(5.428759087785754 46.01008524345922)
2,01004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.958400,5.375901,14359.0,2449.0,POINT(5.375901233977878 45.958399862151865)


In [17]:
df_02 = pd.read_csv(csv_test_02, sep="|", encoding="utf8")
df_02.head(3)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,auteur,logo,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video
0,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,82 avenue Jean Jaurès Directrice du pole Mail:...,• APAJH • Association Charles de Foucauld Cons...,"Afin d’améliorer l’accueil, de plus en plus fr...","Gouvernance, partenariats institutionnels Vie ...",mailto:gwenola.toutain@croix-rouge.fr,Bourgogne,Accompagnement à la parentalité : des acteurs ...,http://www.apriles.net/images/stories/Centrepa...,Centre parental de Migennes - Croix Rouge fran...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: 72 Rue Président Wilson 4...,- Proposer des fruits et légumes frais de sais...,«Relocaliser la production agricole a des impa...,"Gouvernance, partenariats institutionnels Jeun...",http://www.grandcahors.fr mailto:dpennequin@gr...,Midi-Pyrénées,La légumerie : première pierre du Projet alime...,http://www.apriles.net/images/stories/Le%CC%81...,Communauté d'agglomération du Grand Cahors,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"Lien internet: 44100 Nantes Directrice, Pôle é...","Etat (aides à la pierre), Région Pays-de-la-Lo...",Afin de répondre aux besoins d’hébergement de ...,"Petite enfance, modes de garde Parentalité Rel...",mailto:Catherine.bouget@croix-rouge.fr http://...,Pays de la Loire,"Au Bois Bouchaud, parents, enfants et seniors ...",http://www.apriles.net/images/stories/Boisbouc...,Croix Rouge française,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
df_01.columns

Index(['Code_commune_INSEE', 'Nom_commune', 'Code_postal',
       'Libelle_acheminement', 'Ligne_5', 'latitude', 'longitude',
       'Population', 'Surface', 'geopoint_2d'],
      dtype='object')

In [19]:
df_02.columns

Index(['spider_name', 'link_src', 'link_data', 'fiche contact',
       'date du projet', 'partenaires du projet', 'données économiques',
       'résumé du projet', 'tags', 'website', 'auteur', 'logo',
       'adresse du projet', 'titre du projet', 'image(s) du projet',
       'structure porteuse', 'SIRET', 'statut juridique', 'récompenses',
       'video'],
      dtype='object')

In [20]:
### list of meaningfull columns for geoloc
address_columns_01 = ["Nom_commune", "Code_commune_INSEE"]
address_columns_02 = ["adresse du projet"]

In [21]:
addresses = df_01["Nom_commune"].tolist()
print (addresses[:5], "...")

['L ABERGEMENT CLEMENCIAT', 'L ABERGEMENT DE VAREY', 'AMBERIEU EN BUGEY', 'AMBERIEUX EN DOMBES', 'AMBLEON'] ...


In [22]:
### apply concat function to each row (axis=1)
df_01[full_address_col] = df_01.apply(concat_cols, args=[address_columns_01], axis=1)
df_01.head(3)

,Code_commune_INSEE,Nom_commune,Code_postal,Libelle_acheminement,Ligne_5,latitude,longitude,Population,Surface,geopoint_2d,_full_address_
0,01001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.156818,4.924707,767.0,1567.0,POINT(4.924707496412453 46.15681799286022),"L ABERGEMENT CLEMENCIAT, 01001"
1,01002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.010085,5.428759,236.0,909.0,POINT(5.428759087785754 46.01008524345922),"L ABERGEMENT DE VAREY, 01002"
2,01004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.958400,5.375901,14359.0,2449.0,POINT(5.375901233977878 45.958399862151865),"AMBERIEU EN BUGEY, 01004"


In [23]:
df_01.shape

(39201, 11)

In [24]:
### apply concat function to each row (axis=1)
df_02[full_address_col] = df_02.apply(concat_cols, args=[address_columns_02], axis=1)
df_02.sample(3)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,logo,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_
59,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Association « Ensemble demain »...,"Au niveau national, Ensemble Demain est parten...",Afin de permettre aux générations de se rencon...,Relations de voisinage Participation des habit...,http://www.ensembledemain.com mailto:ensembled...,-France entière,Ensemble demain ouvre les portes de l’école,...,Association Ensemble demain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: ACELEM (Association culturelle ...,l'ACELEM réalise une partie de ses actions en ...,Pour lutter contre l'échec scolaire et l'illet...,Participation des habitants Développement urba...,mailto:diemeousman@hotmail.com http://www.acel...,Provence-Alpes-Côte d'Azur,Pour que les livres aient le droit de cité -13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: Association la Main verte...,Education nationale Msa d'Alsace Adapei du Bas...,un guide très complet d’une cinquantaine de pa...,"Accessibilité, Mobilité Emploi, Formation Déve...",mailto:main-verte@wanadoo.fr http://entreprise...,Alsace,La main verte creuse son sillon,...,Association de droit local La Main Verte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df_02.shape

(269, 21)

In [ ]:
### MAIN GEOCODING FUNCTION ON DF 

df_01[location_col] = df_01.apply( 
    geoloc_df, 
    args=[ full_address_col, adress_complement, dft_timeout, dft_delay ], 
    axis=1
)

In [38]:
### MAIN GEOCODING FUNCTION ON DF + SWIFTER

df_02[location_col] = df_02[full_address_col].swifter.apply( 
    geoloc_df_col, 
    complement=adress_complement, 
    #time_out=dft_timeout, 
    #delay=dft_delay, 
)


- row_val :  0                                                   NaN
1                                                   NaN
2                                                   NaN
3                                                   NaN
4                            Centre hospitalier du Mans
5                            Coopérer pour entreprendre
6                               Ville de Mouans-Sartoux
7     Centre de Ressources Politique de la Ville de ...
8                                     Ville de Bordeaux
9                                    ENVIE Autonomie 49
10                                  Association Viltaïs
11                            Ville de Clichy-sous-Bois
12                                        Ville d’Arras
13                                                  NaN
14                                        CCAS d’Angers
15                                                  NaN
Name: _full_address_, dtype: object

- row_val :  nan

- row_val :  nan

- row_val :  nan



- row_val :  Coopérer pour entreprendre
- adress :  Coopérer pour entreprendre, France
- location :  None

- row_val :  Ville de Mouans-Sartoux
- adress :  Ville de Mouans-Sartoux, France
- location :  None

- row_val :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET)
- adress :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET), France
- location :  None

- row_val :  Ville de Bordeaux
- adress :  Ville de Bordeaux, France
- location :  Base sous-marine de Bordeaux, Boulevard Alfred Daney, Bordeaux Maritime, Bordeaux, Gironde, Nouvelle-Aquitaine, France métropolitaine, 33000, France

- row_val :  ENVIE Autonomie 49
- adress :  ENVIE Autonomie 49, France
- location :  None

- row_val :  Association Viltaïs
- adress :  Association Viltaïs, France
- location :  None

- row_val :  Ville de Clichy-sous-Bois
- adress :  Ville de Clichy-sous-Bois, France
- location :  


- row_val :  Coopérer pour entreprendre
- adress :  Coopérer pour entreprendre, France
- location :  None

- row_val :  Ville de Mouans-Sartoux
- adress :  Ville de Mouans-Sartoux, France
- location :  None

- row_val :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET)
- adress :  Centre de Ressources Politique de la Ville de Guyane (Commissariat général à l’égalité des territoires – CGET), France
- location :  None

- row_val :  Ville de Bordeaux
- adress :  Ville de Bordeaux, France
- location :  Base sous-marine de Bordeaux, Boulevard Alfred Daney, Bordeaux Maritime, Bordeaux, Gironde, Nouvelle-Aquitaine, France métropolitaine, 33000, France

- row_val :  ENVIE Autonomie 49
- adress :  ENVIE Autonomie 49, France
- location :  None

- row_val :  Association Viltaïs
- adress :  Association Viltaïs, France
- location :  None

- row_val :  Ville de Clichy-sous-Bois
- adress :  Ville de Clichy-sous-Bois, France
- location :  

Pandas Apply:   0%|          | 0/269 [00:00<?, ?it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val

...


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  Association Energies et jardins du c'ur
- adress :  Association Energies et jardins du c'ur, France
- location :  None


Pandas Apply:  56%|█████▌    | 151/269 [00:42<00:50,  2.33it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  Association la Source
- adress :  Association la Source, France
- location :  Association Pêcheurs Libres, Quai des Anglais, La Source, Saint-Pierre, Martigues, Istres, Bouches-du-Rhône, Provence-Alpes-Côte d'Azur, France métropolitaine, 13500, France


Pandas Apply: 100%|██████████| 269/269 [00:43<00:00,  6.12it/s]


- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val :  nan

- row_val

In [40]:
df_02.sample(10)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_
91,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: Association la Main verte...,Education nationale Msa d'Alsace Adapei du Bas...,un guide très complet d’une cinquantaine de pa...,"Accessibilité, Mobilité Emploi, Formation Déve...",mailto:main-verte@wanadoo.fr http://entreprise...,Alsace,La main verte creuse son sillon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
175,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: Tel: 02.98.46.94.81 29 20...,Ville de Brest ATD quart monde CAF: financemen...,L'arrivée et l'éducation d'un enfant peuvent p...,Participation des habitants Parentalité Protec...,mailto:s.guillemet@adsea29.org http://www.repe...,Bretagne,A Brest : une Maison des Parents - 29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
99,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: Directrice du Foyer Tel: ...,la Région de Bruxelles a donné une prime de ré...,Afin de lutter contre la défiance envers les p...,Vie des séniors Relations de voisinage Partici...,mailto:loredana.marchi@foyer.be http://www.foy...,-Etranger,Deux cultures et trois générations sous le mêm...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
169,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,83000 Toulon Collectif d’Hébergement Varois – ...,-Conseil général du Var -Clic de Toulon (CCAS)...,Afin de favoriser un maintien à domicile de qu...,"Vie des séniors Accessibilité, Mobilité Logeme...",mailto:chv.moissons.nouvelles@gmail.com,Provence-Alpes-Côte d'Azur,Action d'amélioration du logement des personne...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
10,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: Mail: Association Viltaïs 9 Ave...,Parmi les principaux partenaires financiers : ...,"Environ 199 500,00 euros par an, dont près de ...",Afin d’accompagner des jeunes en rupture vers ...,"Gouvernance, partenariats institutionnels Jeun...",mailto:juliette.lucot@viltais.eu http://www.vi...,Auvergne,...,Association Viltaïs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Association Viltaïs,None
152,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,33604 Pessac Cedex Mail: Mairie de Pessac Resp...,Leroy-Merlin Domofrance CCAS Conseil général v...,Afin d'aider les habitants en difficulté finan...,Diagnostic partagé Participation des habitants...,mailto:plateforme@mairie-pessac.fr,Aquitaine,L'atelier économe de Pessac - 33,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
12,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"Lien internet: Mairie d'Arras, Place Guy Molle...",Les équipes pluridisciplinaires de quartier ra...,1. Le Projet de Développement Solidaire d’Arra...,Afin de renforcer l’interconnaissance et la co...,"Gouvernance, partenariats institutionnels Déve...",http://www.arras.fr mailto:a-konieczny@ville-a...,Nord-Pas-de-Calais,...,Ville d’Arras,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville d’Arras,"{'raw': {'place_id': '61791374', 'licence': 'D..."
231,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Associations et organismes de formation parisi...,Afin de favoriser l'apprentissage de la langue...,Intégration Lutte contre l'exclusion sociale,Île-de-France,"A Paris, un ""carnet de santé"" linguistique pou...",Ville de Paris,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
63,Apriles

In [55]:
filtered_df = df_02[df_02[location_col].notnull()].copy()
filtered_df.sample(5)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,adresse du projet,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_
19,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 3 Quai Ceineray Tel: 02 40 99 1...,Le dispositif est entièrement internalisé. Il ...,Financiers : Quatre travailleurs sociaux du Dé...,Afin d’apporter une solution d’accueil adaptée...,Protection de l'enfance Intégration Nouvelles ...,mailto:laurent.gollandeau@loire-atlantique.fr ...,Pays de la Loire,...,Conseil départemental de Loire-Atlantique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil départemental de Loire-Atlantique,"{'raw': {'place_id': '241565046', 'licence': '..."
83,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"21-29, avenue du Général de Gaulle Conseil gén...","Conseil général du Val de Marne, ETAI, AFASER",Financiers : ________________________ 10.000 €...,Afin de développer la participation de tous le...,"Accessibilité, Mobilité Intégration Lutte cont...",http://www.valdemarne.fr mailto:sophie.largeau...,Île-de-France,...,Conseil général du Val de Marne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil général du Val de Marne,"{'raw': {'place_id': '189432783', 'licence': '..."
50,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Assistant de gestion Lien internet: 29000 Quim...,NaN,>Consulter l'article du Journal des acteurs so...,Afin de permettre à de jeunes futurs conducteu...,"Vie des séniors Accessibilité, Mobilité Emploi...",mailto:gmorel@mlpc.asso.fr http://www.mlpc.ass...,Bretagne,...,Mission locale du Pays de Cornouaille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mission locale du Pays de Cornouaille,"{'raw': {'place_id': '61343231', 'licence': 'D..."
22,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 67076 Strasbourg Cedex Mail: Re...,Le projet est désormais porté et piloté par l’...,Une dizaine d’agrès de street workout. Financi...,Afin de favoriser l’insertion de jeunes du qua...,"Jeunesse Développement urbain, Vie des quartie...",mailto:lionel.boizot@strasbourg.eu http://www....,Alsace,...,Eurométropole de Strasbourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eurométropole de Strasbourg,"{'raw': {'place_id': '81368751', 'licence': 'D..."
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,Association la Source,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D..."


In [63]:
def extract_loc(row, field_name="longitude") :
    # print(row)
    # print (type(row))
    return row[field_name]
    
filtered_df["raw"] = filtered_df[location_col].apply(extract_loc, field_name="raw")
filtered_df["lat"] = filtered_df[location_col].apply(extract_loc, field_name="latitude")
filtered_df["long"] = filtered_df[location_col].apply(extract_loc, field_name="longitude")
filtered_df.sample(5)

,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_,raw,lat,long
8,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: chargée de mission handicap et ...,"Préfecture, Direction régionale et département...",• Un chargé de mission analyse des besoins soc...,Afin d’améliorer la coordination et l’interpré...,"Gouvernance, partenariats institutionnels Diag...",http://www.bordeaux.fr mailto:c.beaufort-lance...,Aquitaine,...,NaN,NaN,NaN,NaN,NaN,Ville de Bordeaux,"{'raw': {'place_id': '82004620', 'licence': 'D...","{'place_id': '82004620', 'licence': 'Data © Op...",44.870552,-0.558876
12,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"Lien internet: Mairie d'Arras, Place Guy Molle...",Les équipes pluridisciplinaires de quartier ra...,1. Le Projet de Développement Solidaire d’Arra...,Afin de renforcer l’interconnaissance et la co...,"Gouvernance, partenariats institutionnels Déve...",http://www.arras.fr mailto:a-konieczny@ville-a...,Nord-Pas-de-Calais,...,NaN,NaN,NaN,NaN,NaN,Ville d’Arras,"{'raw': {'place_id': '61791374', 'licence': 'D...","{'place_id': '61791374', 'licence': 'Data © Op...",50.291752,2.779926
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,NaN,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D...","{'place_id': '63721638', 'licence': 'Data © Op...",43.404823,5.055001
22,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 67076 Strasbourg Cedex Mail: Re...,Le projet est désormais porté et piloté par l’...,Une dizaine d’agrès de street workout. Financi...,Afin de favoriser l’insertion de jeunes du qua...,"Jeunesse Développement urbain, Vie des quartie...",mailto:lionel.boizot@strasbourg.eu http://www....,Alsace,...,NaN,NaN,NaN,NaN,NaN,Eurométropole de Strasbourg,"{'raw': {'place_id': '81368751', 'licence': 'D...","{'place_id': '81368751', 'licence': 'Data © Op...",48.574372,7.760331
50,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Assistant de gestion Lien internet: 29000 Quim...,NaN,>Consulter l'article du Journal des acteurs so...,Afin de permettre à de jeunes futurs conducteu...,"Vie des séniors Accessibilité, Mobilité Emploi...",mailto:gmorel@mlpc.asso.fr http://www.mlpc.ass...,Bretagne,...,NaN,NaN,NaN,NaN,NaN,Mission locale du Pays de Cornouaille,"{'raw': {'place_id': '61343231', 'licence': 'D...","{'place_id': '61343231', 'licence': 'Data © Op...",47.994228,-4.091771


,spider_name,link_src,link_data,fiche contact,date du projet,partenaires du projet,données économiques,résumé du projet,tags,website,...,titre du projet,image(s) du projet,structure porteuse,SIRET,statut juridique,récompenses,video,_full_address_,_location_,raw
19,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 3 Quai Ceineray Tel: 02 40 99 1...,Le dispositif est entièrement internalisé. Il ...,Financiers : Quatre travailleurs sociaux du Dé...,Afin d’apporter une solution d’accueil adaptée...,Protection de l'enfance Intégration Nouvelles ...,mailto:laurent.gollandeau@loire-atlantique.fr ...,Pays de la Loire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil départemental de Loire-Atlantique,"{'raw': {'place_id': '241565046', 'licence': '...","{'place_id': '241565046', 'licence': 'Data © O..."
156,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Association la Source Lien internet: Mail: 3 r...,"En novembre 1994, la SCI des Amis de la Poulti...",___________________,>>Consulter la brochure des 20 ans de La Sourc...,Parentalité Lutte contre l'exclusion sociale P...,http://www.associationlasource.fr mailto:rober...,Haute-Normandie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Association la Source,"{'raw': {'place_id': '63721638', 'licence': 'D...","{'place_id': '63721638', 'licence': 'Data © Op..."
83,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,"21-29, avenue du Général de Gaulle Conseil gén...","Conseil général du Val de Marne, ETAI, AFASER",Financiers : ________________________ 10.000 €...,Afin de développer la participation de tous le...,"Accessibilité, Mobilité Intégration Lutte cont...",http://www.valdemarne.fr mailto:sophie.largeau...,Île-de-France,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Conseil général du Val de Marne,"{'raw': {'place_id': '189432783', 'licence': '...","{'place_id': '189432783', 'licence': 'Data © O..."
17,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Fabrique du quartier Danton – Ville du Havre L...,Sur la concertation menée sur le quartier Dant...,700 000 euros investis spécifiquement sur la c...,Afin de mettre les habitants au cœur du projet...,"Rénovation urbaine Développement urbain, Vie d...",mailto:benjamin.lebourg@lehavre.fr http://www....,Haute-Normandie,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ville du Havre,"{'raw': {'place_id': '22957963', 'licence': 'D...","{'place_id': '22957963', 'licence': 'Data © Op..."
22,Apriles,http://www.apriles.net/index.php?option=com_so...,http://www.apriles.net/index.php?option=com_so...,Lien internet: 67076 Strasbourg Cedex Mail: Re...,Le projet est désormais porté et piloté par l’...,Une dizaine d’agrès de street workout. Financi...,Afin de favoriser l’insertion de jeunes du qua...,"Jeunesse Développement urbain, Vie des quartie...",mailto:lionel.boizot@strasbourg.eu http://www....,Alsace,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Eurométropole de Strasbourg,"{'raw': {'place_id': '81368751', 'licence': 'D...","{'place_id': '81368751', 'licence': 'Data © Op..."


In [ ]:
### cf : https://geopy.readthedocs.io/en/stable/#usage-with-pandas
geocoder_nom = Nominatim(user_agent="_solidata_app_")
geocoder_ban = BANFrance(user_agent="_solidata_app_")

### rate limiter
geocode_nom = RateLimiter(geocoder_nom.geocode, min_delay_seconds=dft_delay)
geocode_ban = RateLimiter(geocoder_ban.geocode, min_delay_seconds=dft_delay)

In [ ]:
df_01[location_col+"_nom"] = df_01[full_address_col].apply(partial(geocode_nom, language='en'))

In [ ]:
df_02[location_col+"_ban"] = df_02[full_address_col].progress_apply(geocode_ban)
# df_02[location_col+"_ban"] = df_02[full_address_col].apply(geocode_ban)

In [ ]:
df_02.sample(10)